<a href="https://colab.research.google.com/github/brianellis1997/Sarcasm_Detection/blob/Elena/DS_440_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Detection using NLP Techniques
### Team 2: Brian Ellis, Lindsey Rich, Elena Kern

## Load Libraries and Dataset

In [8]:
!pip install transformers

In [9]:
import pandas as pd
import torch
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#test comment

In [10]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [11]:
!git clone https://github.com/brianellis1997/Sarcasm_Detection.git

Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 239 (delta 134), reused 115 (delta 62), pack-reused 3
Receiving objects: 100% (239/239), 7.20 MiB | 7.56 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_bal = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Train_Balanced.csv')   # Make sure path is correct in your google drive
train_bal.head()

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,522142,0,"I personally wasn't a huge Garrosh fan, I've a...",cromemako83,AskReddit,2,2,0,2015-07-01,2015-07-11 01:55:53,Fuck Vol'jin. Garrosh Hellscream did nothing w...
1,907864,1,you forgot the,_SharkWeek_,AskReddit,1,1,0,2013-03-01,2013-03-14 03:03:46,That's a lie fed to you by the LIEberal media....
2,604170,1,"Nah man, she's clearly an ad carry",jdswift13,leagueoflegends,1,1,0,2015-10-01,2015-10-21 23:22:17,she isnt already?
3,110635,1,This sub in a nutshell.,trickz-M-,GlobalOffensive,1,-1,-1,2016-12-01,2016-12-05 03:50:18,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(
4,997758,0,Yes... I do.,guriboysf,videos,4,4,0,2010-01-01,2010-01-17 21:32:40,"""so, i hear you have a fat cock."""


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Test_Balanced.csv')

In [ ]:
train_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      808618 non-null  int64 
 1   label           808618 non-null  int64 
 2   comment         808618 non-null  object
 3   author          808618 non-null  object
 4   subreddit       808618 non-null  object
 5   score           808618 non-null  int64 
 6   ups             808618 non-null  int64 
 7   downs           808618 non-null  int64 
 8   date            808618 non-null  object
 9   created_utc     808618 non-null  object
 10  parent_comment  808618 non-null  object
dtypes: int64(5), object(6)
memory usage: 67.9+ MB


In [ ]:
train_bal['label'].value_counts()

1    404369
0    404249
Name: label, dtype: int64

We can see our dataset is balanced.

In [ ]:
train_bal['label'].value_counts(normalize=True)

1    0.500074
0    0.499926
Name: label, dtype: float64

## Train Logit with text and non-text covars

In [13]:
import pandas as pd

train_bal = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Train_Balanced.csv')   # Make sure path is correct in your google drive
train_bal.head()

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,522142,0,"I personally wasn't a huge Garrosh fan, I've a...",cromemako83,AskReddit,2,2,0,2015-07-01,2015-07-11 01:55:53,Fuck Vol'jin. Garrosh Hellscream did nothing w...
1,907864,1,you forgot the,_SharkWeek_,AskReddit,1,1,0,2013-03-01,2013-03-14 03:03:46,That's a lie fed to you by the LIEberal media....
2,604170,1,"Nah man, she's clearly an ad carry",jdswift13,leagueoflegends,1,1,0,2015-10-01,2015-10-21 23:22:17,she isnt already?
3,110635,1,This sub in a nutshell.,trickz-M-,GlobalOffensive,1,-1,-1,2016-12-01,2016-12-05 03:50:18,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(
4,997758,0,Yes... I do.,guriboysf,videos,4,4,0,2010-01-01,2010-01-17 21:32:40,"""so, i hear you have a fat cock."""


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

train_bal['date'] = pd.to_datetime(train_bal['date']) # Run this to convert date to date_time
train_bal['created_utc'] = pd.to_datetime(train_bal['created_utc']) # Run this to convert created_utc to date_time
train_bal['hour'] = train_bal['created_utc'].dt.hour # Run this to feature engineer hour variable
train_bal['month'] = train_bal['date'].dt.month # Run this cell to add month and day of week variables
train_bal['dow'] = train_bal['date'].dt.day_of_week
train_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      808618 non-null  int64         
 1   label           808618 non-null  int64         
 2   comment         808618 non-null  object        
 3   author          808618 non-null  object        
 4   subreddit       808618 non-null  object        
 5   score           808618 non-null  int64         
 6   ups             808618 non-null  int64         
 7   downs           808618 non-null  int64         
 8   date            808618 non-null  datetime64[ns]
 9   created_utc     808618 non-null  datetime64[ns]
 10  parent_comment  808618 non-null  object        
 11  hour            808618 non-null  int64         
 12  month           808618 non-null  int64         
 13  dow             808618 non-null  int64         
dtypes: datetime64[ns](2), int64(8), obje

In [15]:
# Drop unneeded columns
train = train_bal.drop(columns=['Unnamed: 0', 'date', 'created_utc', 'author'])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   label           808618 non-null  int64 
 1   comment         808618 non-null  object
 2   subreddit       808618 non-null  object
 3   score           808618 non-null  int64 
 4   ups             808618 non-null  int64 
 5   downs           808618 non-null  int64 
 6   parent_comment  808618 non-null  object
 7   hour            808618 non-null  int64 
 8   month           808618 non-null  int64 
 9   dow             808618 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 61.7+ MB


In [16]:
X = train.drop(columns=['label', 'comment', 'parent_comment', 'subreddit'])
y = train['label']

In [17]:
sc = StandardScaler() # Scale variables

X = sc.fit_transform(X)
X = pd.DataFrame(X)
train_cat = pd.concat([train, X], axis=1)

In [18]:
train_cat.drop(columns=['score', 'ups', 'downs', 'hour', 'month', 'dow', 'label', 'subreddit'], inplace=True)
# train_cat.rename(columns={'0':'score', '1':'ups', '2':'downs', '3':'hour', '4':'month', '5':'dow'}, inplace=True)
train_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808618 entries, 0 to 808617
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   comment         808618 non-null  object 
 1   parent_comment  808618 non-null  object 
 2   0               808618 non-null  float64
 3   1               808618 non-null  float64
 4   2               808618 non-null  float64
 5   3               808618 non-null  float64
 6   4               808618 non-null  float64
 7   5               808618 non-null  float64
dtypes: float64(6), object(2)
memory usage: 49.4+ MB


In [ ]:
# subreddit_counts = train_bal.groupby('subreddit').size().sort_values(ascending=False)
# subreddit_counts.head()

In [ ]:
# Calculate the 5% quantile
# quantile_5 = subreddit_counts.quantile(0.992)

# # Select subreddits whose counts are greater than or equal to the 5% quantile
# top_subreddits = subreddit_counts[subreddit_counts >= quantile_5]
# top_subreddits.size # 680 subreddits in the top 5%

In [ ]:
# subreddits = top_subreddits.index.tolist()
# subreddits[:5]

In [ ]:
# def filter_subreddit(subreddit):
#     if subreddit in subreddits:
#         return subreddit
#     else:
#         return 'other'

# # Apply the filter_subreddit function to the 'subreddit' column
# train['subreddit'] = train['subreddit'].apply(filter_subreddit)

# train['subreddit'].nunique()

In [ ]:
# top_subreddits = train['subreddit'].value_counts(ascending=False).head(10).index
# subreddit_filter = train[train['subreddit'].isin(top_subreddits)]

# subreddit_filter.info()

In [ ]:
# One-hot encode subreddit
# subreddit_dummies = pd.get_dummies(train['subreddit'])

In [ ]:
# # Combine one-hot encoded categorical variables with numeric variables
# train_encoded = pd.concat([train_cat, subreddit_dummies], axis=1)
# train_encoded.info()

In [ ]:
# train_encoded.drop(columns=['subreddit'], inplace=True)

In [19]:
train_encoded = train_cat
train_encoded.shape

(808618, 8)

In [20]:
train_encoded.head()

,comment,parent_comment,0,1,2,3,4,5
0,"I personally wasn't a huge Garrosh fan, I've a...",Fuck Vol'jin. Garrosh Hellscream did nothing w...,-0.101353,-0.084359,0.413395,-1.581123,-0.032928,-0.461911
1,you forgot the,That's a lie fed to you by the LIEberal media....,-0.122055,-0.108421,0.413395,-1.308629,-1.205315,0.556223
2,"Nah man, she's clearly an ad carry",she isnt already?,-0.122055,-0.108421,0.413395,1.416313,0.846362,0.047156
3,This sub in a nutshell.,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(,-0.122055,-0.156544,-2.418994,-1.308629,1.432556,0.047156
4,Yes... I do.,"""so, i hear you have a fat cock.""",-0.059948,-0.036236,0.413395,1.143819,-1.791509,0.556223


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack

tf_idf_comment = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)
tf_idf_parent_comment = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)

comment_vectors = tf_idf_comment.fit_transform(train_encoded['comment'])
parent_comment_vectors = tf_idf_parent_comment.fit_transform(train_encoded['parent_comment'])

# Concatenate the TF-IDF vectors horizontally
combined_vectors = hstack([comment_vectors, parent_comment_vectors])
print(combined_vectors.shape)

(808618, 1878463)


## Stacking with parent comment

In [21]:
# Get other variables from train_encoded
other_variables = train_encoded.drop(columns=['comment', 'parent_comment'])

# Concatenate other variables horizontally with TF-IDF vectors
stacked_vars = hstack([combined_vectors, other_variables])
print(stacked_vars.shape)

NameError: name 'hstack' is not defined

In [ ]:
# Split the data into train and test sets
#X_train, X_val, y_train, y_val = train_test_split(stacked_vars, y, test_size=0.2, random_state=42)

#logit = LogisticRegression(C=1, n_jobs=-1, solver='lbfgs',
#                                  random_state=22, verbose=1)

#logit.fit(X_train, y_train)
#y_pred = logit.predict(X_val)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
from sklearn.decomposition import TruncatedSVD
# Split the data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(combined_vectors, y, test_size=0.2, random_state=42)

svd = TruncatedSVD(n_components=100, random_state=42)
X_train_svd = svd.fit_transform(X_train)
X_val_svd = svd.transform(X_val)

svm_model = SVC(kernel='linear', random_state=42)

svm_model.fit(X_train_svd, y_train)

y_pred = svm_model.predict(X_val_svd)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_val, y_pred)
print('Accuracy: %f' % accuracy)

precision = precision_score(y_val, y_pred)
print('Precision: %f' % precision)

recall = recall_score(y_val, y_pred)
print('Recall: %f' % recall)

f1 = f1_score(y_val, y_pred)
print('F1 score: %f' % f1)

auc = roc_auc_score(y_val, y_pred)
print('ROC AUC: %f' % auc)

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.decomposition import TruncatedSVD

# Split the data into train and test sets, just using comment and parent comment for this case
X_train, X_val, y_train, y_val = train_test_split(combined_vectors, y, test_size=0.2, random_state=42)

svd = TruncatedSVD(n_components=100, random_state=42)
X_train_svd = svd.fit_transform(X_train)
X_val_svd = svd.transform(X_val)
print(X_train_svd.shape)

dtrain = xgb.DMatrix(X_train_svd, label=y_train)
dval = xgb.DMatrix(X_val_svd, label=y_val)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'scale_pos_weight': 1
}

num_rounds = 50

# Training the model
xgb_model = xgb.train(params, dtrain, num_rounds, evals=[(dval, "Validation")])

# Predict on the validation set
y_pred_xgb = xgb_model.predict(dval)

# Convert predicted probabilities to binary predictions
y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred_xgb]


(646894, 100)
[0]	Validation-logloss:0.69099
[1]	Validation-logloss:0.68863
[2]	Validation-logloss:0.68660
[3]	Validation-logloss:0.68497
[4]	Validation-logloss:0.68346
[5]	Validation-logloss:0.68199
[6]	Validation-logloss:0.68073
[7]	Validation-logloss:0.67955
[8]	Validation-logloss:0.67854
[9]	Validation-logloss:0.67755
[10]	Validation-logloss:0.67665
[11]	Validation-logloss:0.67580
[12]	Validation-logloss:0.67503
[13]	Validation-logloss:0.67436
[14]	Validation-logloss:0.67362
[15]	Validation-logloss:0.67295
[16]	Validation-logloss:0.67232
[17]	Validation-logloss:0.67183
[18]	Validation-logloss:0.67130
[19]	Validation-logloss:0.67076
[20]	Validation-logloss:0.67031
[21]	Validation-logloss:0.66982
[22]	Validation-logloss:0.66940
[23]	Validation-logloss:0.66889
[24]	Validation-logloss:0.66844
[25]	Validation-logloss:0.66798
[26]	Validation-logloss:0.66755
[27]	Validation-logloss:0.66711
[28]	Validation-logloss:0.66678
[29]	Validation-logloss:0.66636
[30]	Validation-logloss:0.66599
[31]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_val, y_pred_binary)
print('Accuracy: %f' % accuracy)

precision = precision_score(y_val, y_pred_binary)
print('Precision: %f' % precision)

recall = recall_score(y_val, y_pred_binary)
print('Recall: %f' % recall)

f1 = f1_score(y_val, y_pred_binary)
print('F1 score: %f' % f1)

auc = roc_auc_score(y_val, y_pred_binary)
print('ROC AUC: %f' % auc)

Accuracy: 0.600035
Precision: 0.620491
Recall: 0.517412
F1 score: 0.564283
ROC AUC: 0.600126


In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased')

# Example sentences (change these to your own data)
sentences = [
    "I LOVE it when my computer crashes.",
    "Yeah, because sarcasm is really hard to detect."
]

# Tokenize the sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Perform inference
outputs = model(**inputs)

# Get the predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1)

# Print the predicted labels
for sentence, label in zip(sentences, predicted_labels.tolist()):
    print(f"Sentence: {sentence}\nPredicted Label: {label}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentence: I LOVE it when my computer crashes.
Predicted Label: 1
Sentence: Yeah, because sarcasm is really hard to detect.
Predicted Label: 0
